#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 6: Fashion MNIST  </center>

En esta actividad se diseñará y entrenará una arquitectura para clasificar las imágenes del dataset *Fashion MNIST*.

## Objetivos

 - Adquirir experiencia práctica en la implementación de redes neuronales con la biblioteca *keras*
 - Evaluar la influencia que tienen en el entrenamiento de una *red profunda* la inclusión de técnicas como *Batch Normalization* o *Dropout* en la arquitectura de la red.
 - Evaluar el impacto de la *inicialización* de los pesos en el entrenamiento de una *red profunda*.
 - Diseñar, entrenar y evaluar arquitecturas para clasificar imágenes.  


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2025/blob/main/talleres/taller6_Fashion-MNIST.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

In [1]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [2]:
from comet_ml import Experiment
import tensorflow as tf
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
*!pip install paquete_faltante*

### Parte 1 - Levantar los datos

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Se trabajará con el conjunto de datos *Fashion MNIST* disponible en [*Keras*](https://keras.io/api/datasets/fashion_mnist/). Levantar los datos y separar, de los datos disponibles para entrenamiento, un subconjunto de 10000 muestras para validación.

### Parte 2  - Exploración de datos

Como es habitual, una buena práctica es explorar los datos para familiarizarse con el problema. En este caso, además de en keras, el *dataset* está disponible en *Tensorflow Datasets*.

Conteste a las siguientes preguntas:        
    - ¿Cuántas imágenes hay disponibles? ¿De qué tamaño son?       
    - ¿Cuál es el tipo de dato? ¿Es adecuado?         
Muestre un ejemplo de cada clase.

In [6]:
print(x_train.shape)
print(x_train.dtype)


(60000, 28, 28)
uint8


### Parte 3 - Un primer modelo

Construya una red neuronal totalmente conectada de 20 capas y 100 neuronas por capa. Para ello se sugiere completar la implementación de la función `fully_connected_model`. Utilice *relu* como función de activación y *sgd* como optimizador.  Indique cuántos parámetros tiene la red.

In [18]:
def fully_connected_model(input_shape, n_hiddens, n_neurons, activation='relu',
                        optimizer='sgd', learning_rate = 1e-3):

    '''
    Entrada:
        input_shape: [M,N]
        n_hiddens: número de capas ocultas
        n_neurons: número de neuronas en cada capa oculta
        activation: función de activación de las neuronas. Por defecto 'relu'.
        optimizer: método de optimización. Por defecto 'sgd'.
        learning_rate: tasa de aprendizaje del optimizador. Por defecto 1e-3.
    Salida:
        model: modelo generado
    '''
    tf.random.set_seed(42)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hiddens):
        model.add(tf.keras.layers.Dense(n_neurons, activation=activation))



    if optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Optimizador '{optimizer}' no soportado.")

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']
               )

    return model

### Parte 4 -  Un primer entrenamiento

In [22]:
x_train_norm, x_test_norm = x_train / 255., x_test / 255.

Entrene el modelo generado en la parte anterior durante 10 épocas para los siguientes valores de *learning rate*: [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]. Muestre los desempeños en los conjuntos de entrenamiento y validación.

In [24]:
model = fully_connected_model(input_shape=[28, 28], n_hiddens=20, n_neurons=100)
model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1020 - loss: 8.1150 - val_accuracy: 0.1000 - val_loss: 7.9453
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.1028 - loss: 7.8917 - val_accuracy: 0.1000 - val_loss: 7.9452
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1028 - loss: 7.8915 - val_accuracy: 0.1000 - val_loss: 7.9449
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1028 - loss: 7.8912 - val_accuracy: 0.1000 - val_loss: 7.9443
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.1028 - loss: 7.8905 - val_accuracy: 0.1000 - val_loss: 7.9434
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.1028 - loss: 7.8895 - val_accuracy: 0.1000 - val_loss: 7.9414
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1028 - loss: 7.8869 - val_accuracy: 0.1000 - val_loss: 7.9355
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.1028 - loss: 7.8773

In [13]:
model = fully_connected_model(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=1e-4)
model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1144 - loss: 4.7445 - val_accuracy: 0.1871 - val_loss: 2.0086
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.3226 - loss: 1.8544 - val_accuracy: 0.4336 - val_loss: 1.5211
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.3974 - loss: 1.7872 - val_accuracy: 0.4361 - val_loss: 1.5030
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.4436 - loss: 2.1205 - val_accuracy: 0.1000 - val_loss: 10.8334
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.1003 - loss: 9.8259 - val_accuracy: 0.1000 - val_loss: 6.2164
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1003 - loss: 6.2079 - val_accuracy: 0.1000 - val_loss: 6.0970
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1003 - loss: 6.0654 - val_accuracy: 0.1000 - val_loss: 5.9327
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.1003 - loss: 5.925

In [14]:
model = fully_connected_model(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=5e-4)
model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0903 - loss: 9.6497 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1004 - loss: 10.4426 - val_accuracy: 0.1000 - val_loss: 10.3791
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1004 -

In [15]:
model = fully_connected_model(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=5e-3)
model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1012 - loss: 8.8041 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1016 - loss: 8.8161 - val_accuracy: 0.1000 - val_loss: 8.7867
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.1016 - loss: 8.81

In [16]:
model = fully_connected_model(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=1e-2)
model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1487 - loss: 3.5091 - val_accuracy: 0.2375 - val_loss: 3.4979
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.2087 - loss: 5.5596 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.0991 - loss: 8.3988 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 8.3988 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 8.3988 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.0991 - loss: 8.3988 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.0991 - loss: 8.3988 - val_accuracy: 0.1000 - val_loss: 8.3618
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 8.398

### Parte 5: - Cambio de Inicialización

**5a)** Repita el experimento pero cambiando el método de inicialización de los pesos. En vez de utilizar la inicialización por defecto, utilice 'he_normal'. Comente como cambian los resultados.

In [20]:
def fully_connected_model_kernel(input_shape, n_hiddens, n_neurons, activation='relu',
                        optimizer='sgd', learning_rate = 1e-3, kernel_initializer="he_normal" ):

    '''
    Entrada:
        input_shape: [M,N]
        n_hiddens: número de capas ocultas
        n_neurons: número de neuronas en cada capa oculta
        activation: función de activación de las neuronas. Por defecto 'relu'.
        optimizer: método de optimización. Por defecto 'sgd'.
        learning_rate: tasa de aprendizaje del optimizador. Por defecto 1e-3.
    Salida:
        model: modelo generado
    '''
    tf.random.set_seed(42)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hiddens):
        model.add(tf.keras.layers.Dense(n_neurons, activation=activation, kernel_initializer=kernel_initializer))



    if optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Optimizador '{optimizer}' no soportado.")

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']
               )

    return model

In [21]:
model = fully_connected_model_kernel(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=1e-4)
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0741 - loss: 12.4246 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.0991 - loss: 13.5284 - val_accuracy: 0.1000 - val_loss: 13.5210
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.0991 

In [23]:
model = fully_connected_model_kernel(input_shape=[28, 28], n_hiddens=20, n_neurons=100, learning_rate=1e-4)
history = model.fit(x_train_norm, y_train, epochs=10,
                    validation_data=(x_test_norm, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.1097 - loss: 6.1525 - val_accuracy: 0.3841 - val_loss: 1.8159
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.3580 - loss: 2.2701 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.0991 - loss: 8.5851 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 8.5851 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.0991 - loss: 8.5851 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.0991 - loss: 8.5851 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0991 - loss: 8.5851 - val_accuracy: 0.1000 - val_loss: 8.5731
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.0991 - loss: 8.5851

**5b) (opcional):** Probar otras formas de inicialización. Por ejemplo, ¿Qué pasa si se utiliza 'random_normal'?

### Parte 6 - Batch Normalization

Modifique la implementación de la función *fully_connected_model* para que admita la posibilidad de agregar capas de *batch normalization*. En caso de utilizar *batch normalization* coloque dichas capas luego de las capas densas y antes de las activaciones, tal como fue propuesto en el [paper original](https://arxiv.org/pdf/1502.03167.pdf).

**6a)** ¿Cuántos parámetros tiene el modelo con *batch normalization*? ¿Qué son y para que se usan los *Non-trainable-parameters*?

**6b)** Repita el experimento de la **parte 4** pero ahora utilizando el modelo con *batch normalization*. Para este experimento vuelva a la inicialización de los pesos por defecto ('glorot_uniform').

### Parte 7 -  Optimizador

**Parte 7a)** De ser necesario, modifique la implementación de la función `fully_connected_model()` para que permita optimizar utilizando el método *adam*. Repita el experimento de la **parte 4** pero modificando solamente el optimizador. Comente los cambios que observa.

**Parte 7b)** ¿Cambian los resultados del entrenamiento si en la parte anterior además de cambiar el optimizador se agrega *batch normalization* ?

### Parte 8  -  Red Neuronal Convolucional

**8a)** Escribir una función similar a la de la **parte 3** pero que construya una red neuronal convolucional con la siguiente arquitectura:
    
N * [Conv2D --> Conv2D --> Pool2D] → Flatten → Dense → Dense.

donde *N* representa la cantidad de bloques [Conv2D --> Conv2D --> Pool2D] a utilizar.

**8b)** Entrenar una realización de la red. Por ejemplo, utilizar:
- N=2
- 64 filtros de convolución de tamaño 3 en la primera capa del bloque
- 32 filtros de convolución de tamaño 3 en la segunda capa del bloque
- 128 neuronas en la primera capa densa

**8c)** ¿Cuántos parámetros tiene el modelo?

### Parte 9 - Entrenar la mejor arquitectura posible

En esta parte el objetivo será entrenar el mejor clasificador posible para *Fashion MNIST*. La única restricción es que el modelo de arquitectura tendrá que ser como el de la **parte 3** o como el de la **parte 8**. En ambos casos se podrán evaluar las siguientes variaciones:     
    - Número de capas y neuronas por capa en el modelo de la **parte3**     
    - Número de bloques, cantidad de filtros de bloque y tamaño de los filtros en el caso del modelo de la **parte 8**              
    - Distintas funciones de activación y métodos de inicialización de pesos     
    - Agregado de *Batch Normalization* y/o *Dropout*    
    - Distintos métodos de optimización y sus parámetros     

Para entrenar los modelos se recomienda la utilización de alguno de los [callbacks](https://keras.io/callbacks/) disponibles en *keras*. Por ejemplo: *early stopping* y *model checkpoint*.